In [3]:
import sys
sys.path.append('../..')
from datasets import load_dataset, load_dataset_builder

from astropile.utils import cross_match_datasets
from photo_z_wrapper import PhotoZWrapper

ImportError: cannot import name 'PhotoZWrapper' from 'photo_z_wrapper' (/mnt/home/lparker/Documents/AstroFoundationModel/AstroPile_prototype/baselines/photo_z/photo_z_wrapper.py)

In [ ]:
# Load Dataset Builders 
hsc_builder = load_dataset_builder('/mnt/ceph/users/polymathic/AstroPile_tiny/hsc/hsc.py', trust_remote_code=True)
desi_builder = load_dataset_builder('/mnt/ceph/users/polymathic/AstroPile_tiny/desi/desi.py', trust_remote_code=True)

# Cross-Match Datasets with AstroPile
hsc_meets_desi = cross_match_datasets(desi_builder, hsc_builder,
                                      matching_radius=1.0,
                                      keep_in_memory=True,
                                      )

hsc_meets_desi.set_format('torch')

/mnt/home/lparker/python_envs/astrokernel/lib/python3.9/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Initial number of matches:  1286
Number of matches lost at healpix region borders:  0
Final size of cross-matched catalog:  1286


In [ ]:
sys.path.append('../')
from utils import split_dataset

# Use prebuilt split function to split the dataset (currently supports naive)
train_dataset, test_dataset = split_dataset(hsc_meets_desi, split='naive')

In [ ]:
# Create PhotoZWrapper for training and testing
photo_z = PhotoZWrapper(
    train_dataset,
    test_dataset, 
    feature_flag='image.array', # feature flag
    label_flag='Z',             # label flag
    feature_dynamic_range=True,
    label_dynamic_range=False,
    feature_z_score=True,
    label_z_score=True,
    loading='iterated',         # iterated or full
    batch_size=128, 
    num_workers=16, 
    test_size=0.2, 
    )    

In [ ]:
# Prepare data before using the data loader
# This computes the statistics and sets up the collate functions correctly
photo_z.prepare_data()


Computing statistics for image.array:   0%|          | 0/9 [00:00<?, ?it/s]

Computing statistics for Z: 100%|██████████| 9/9 [00:01<00:00,  4.94it/s]


In [ ]:
train_loader = photo_z.train_dataloader()
val_loader = photo_z.val_dataloader()
test_loader = photo_z.test_dataloader()

In [ ]:
x, y = next(iter(train_loader))

(torch.Size([128, 5, 144, 144]), torch.Size([5, 1, 128]))

In [2]:
import matplotlib.pyplot as plt
plt.imshow(x[5,:3,:,:].T)

NameError: name 'x' is not defined